### Loading Datasets (Cells 11, 19–24);
Loaeds five datasets (prices, generation, consumption, forecast generation, and cross-border flows) from CSV files using pandas.
 Specifids semicolon (`;`) as the delimiter and parsds the `Start date` column as datetime.
 Handlds mixed data types with warnings suppressed using `low_memory=Falsey.

In [11]:
prices = pd.read_csv(
    r"C:\Users\HP\Documents\PowercastLunar\Day-ahead_prices_202301010000_202503050000_Hour.csv",
    delimiter=";",  
    parse_dates=["Start date"]
)

C:\Users\HP\AppData\Local\Temp\ipykernel_7352\3405506440.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices = pd.read_csv(


In [19]:
generation = pd.read_csv(
    r"C:\Users\HP\Documents\PowercastLunar\Actual_generation_202301010000_202503050000_Quarterhour.csv",
    delimiter=";",
    parse_dates=["Start date"]
)
generation = clean_column_names(generation)
# Rename ambiguous columns 
generation.rename(columns={"other_conventional_mwh": "fossil_other_mwh"}, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_7352\1188790165.py:1: DtypeWarning: Columns (3,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  generation = pd.read_csv(
C:\Users\HP\AppData\Local\Temp\ipykernel_7352\1188790165.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  generation = pd.read_csv(


In [20]:
consumption = pd.read_csv(
    r"C:\Users\HP\Documents\PowercastLunar\Actual_consumption_202301010000_202503050000_Quarterhour.csv",
    delimiter=";",
    parse_dates=["Start date"]
)
consumption = clean_column_names(consumption)

C:\Users\HP\AppData\Local\Temp\ipykernel_7352\1789502006.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  consumption = pd.read_csv(


In [23]:
forecast_gen = pd.read_csv(
    r"C:\Users\HP\Documents\PowercastLunar\Forecasted_generation_Day-Ahead_202301010000_202503050000_Hour_Quarterhour.csv",
    delimiter=";",
    parse_dates=["Start date"]
)
forecast_gen = clean_column_names(forecast_gen)

C:\Users\HP\AppData\Local\Temp\ipykernel_7352\1803036387.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast_gen = pd.read_csv(


In [24]:
cross_border = pd.read_csv(
    r"C:\Users\HP\Documents\PowercastLunar\Cross-border_physical_flows_202301010000_202503050000_Quarterhour.csv",
    delimiter=";",
    parse_dates=["Start date"]
)
cross_border = clean_column_names(cross_border)
# Rename columns for clarity 
cross_border.rename(columns={"net_export_mwh_original_resolutions": "net_export_mwh"}, inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_7352\3964097221.py:1: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  cross_border = pd.read_csv(
C:\Users\HP\AppData\Local\Temp\ipykernel_7352\3964097221.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cross_border = pd.read_csv(


### Cleaning Column Names
* Standardized column names across datasets by converting to lowercase, replacing special characters (e.g., `[`, `]`, `€`) with underscores. 
* Removed unnecessary suffixes like "original resolutions".
* Ensured consistency for merging.

In [15]:
def clean_column_names(df):
    """Standardize column names (lowercase, underscores, remove units)."""
    df.columns = (
        df.columns.str.lower()
        .str.replace(r"[\[\]€/]", "", regex=True)
        .str.replace("original resolutions", "", case=False)
        .str.replace(" ", "_")
        .str.strip("_")
    )
    return df

prices = clean_column_names(prices)

In [17]:
print(prices.columns.tolist())

['start_date', 'end_date', 'germanyluxembourg_mwh', '∅_delu_neighbours_mwh', 'belgium_mwh', 'denmark_1_mwh', 'denmark_2_mwh', 'france_mwh', 'netherlands_mwh', 'norway_2_mwh', 'austria_mwh', 'poland_mwh', 'sweden_4_mwh', 'switzerland_mwh', 'czech_republic_mwh', 'deatlu_mwh', 'northern_italy_mwh', 'slovenia_mwh', 'hungary_mwh']


### Processing Quarter-Hourly Data* Resampleds quarter-hourly datasets (generation, consumption, cross-border) to hourly intervals.
*  Convereds `start_date` to datetime with a specific format (`%b %d, %Y %I:%M %p` 
*  dropeds non-numeric``end_dat` .
*  convereds remaining columns to numeric (coercing errors to NaN
*  aggregatd  using mean values per hour.

In [57]:
import pandas as pd

def clean_column_names(df):
    """Standardize column names (lowercase, underscores, remove units/symbols)."""
    df.columns = (
        df.columns.str.lower()
        .str.replace(r"[\[\]€/∅()]", "", regex=True)
        .str.replace("original resolutions", "", case=False)
        .str.replace(" ", "_")
        .str.strip("_")
    )
    return df

def process_dataset(df):
    """Process individual dataset with strict type handling."""
    # Convert dates
    df["start_date"] = pd.to_datetime(
        df["start_date"],
        format="%b %d, %Y %I:%M %p",
        errors="coerce"
    )
    df = df.dropna(subset=["start_date"])
    
    # Remove non-numeric columns 
    df = df.drop(columns=["end_date"], errors="ignore")
    
    # Convert all remaining columns to numeric
    for col in df.columns:
        if col != "start_date":
            df[col] = pd.to_numeric(df[col], errors="coerce")
    
    # Drop columns that failed conversion (all NaN)
    df = df.dropna(axis=1, how="all")
    
    # Resample
    return (
        df.set_index("start_date")
        .resample("H")
        .mean(numeric_only=True)  
        .reset_index()
    )

# Load and process datasets
data_paths = {
    "prices": r"C:\Users\HP\Documents\PowercastLunar\Day-ahead_prices_202301010000_202503050000_Hour.csv",
    "generation": r"C:\Users\HP\Documents\PowercastLunar\Actual_generation_202301010000_202503050000_Quarterhour.csv",
    "consumption": r"C:\Users\HP\Documents\PowercastLunar\Actual_consumption_202301010000_202503050000_Quarterhour.csv",
    "cross_border": r"C:\Users\HP\Documents\PowercastLunar\Cross-border_physical_flows_202301010000_202503050000_Quarterhour.csv"
}

datasets = {}
for name, path in data_paths.items():
    print(f"\n=== Processing {name} ===")
    
    # Load data
    df = pd.read_csv(path, delimiter=";")
    df = clean_column_names(df)
    
    # Process quarter-hourly data
    if name != "prices":
        df = process_dataset(df)
    
    datasets[name] = df
    print(f"Processed {name} with columns:", df.columns.tolist())

print("\nAll datasets processed successfully!")


=== Processing prices ===
Processed prices with columns: ['start_date', 'end_date', 'germanyluxembourg_mwh', 'delu_neighbours_mwh', 'belgium_mwh', 'denmark_1_mwh', 'denmark_2_mwh', 'france_mwh', 'netherlands_mwh', 'norway_2_mwh', 'austria_mwh', 'poland_mwh', 'sweden_4_mwh', 'switzerland_mwh', 'czech_republic_mwh', 'deatlu_mwh', 'northern_italy_mwh', 'slovenia_mwh', 'hungary_mwh']

=== Processing generation ===


C:\Users\HP\AppData\Local\Temp\ipykernel_7352\454245991.py:56: DtypeWarning: Columns (3,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";")


Processed generation with columns: ['start_date', 'biomass_mwh', 'hydropower_mwh', 'wind_offshore_mwh', 'wind_onshore_mwh', 'photovoltaics_mwh', 'other_renewable_mwh', 'nuclear_mwh', 'lignite_mwh', 'hard_coal_mwh', 'fossil_gas_mwh', 'hydro_pumped_storage_mwh', 'other_conventional_mwh']

=== Processing consumption ===
Processed consumption with columns: ['start_date', 'residual_load_mwh', 'hydro_pumped_storage_mwh']

=== Processing cross_border ===


C:\Users\HP\AppData\Local\Temp\ipykernel_7352\454245991.py:56: DtypeWarning: Columns (5,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";")


Processed cross_border with columns: ['start_date', 'net_export_mwh', 'netherlands_export_mwh', 'netherlands_import_mwh', 'switzerland_export_mwh', 'switzerland_import_mwh', 'denmark_export_mwh', 'denmark_import_mwh', 'czech_republic_export_mwh', 'czech_republic_import_mwh', 'luxembourg_export_mwh', 'luxembourg_import_mwh', 'sweden_export_mwh', 'sweden_import_mwh', 'austria_export_mwh', 'austria_import_mwh', 'france_export_mwh', 'france_import_mwh', 'poland_export_mwh', 'poland_import_mwh', 'norway_export_mwh', 'norway_import_mwh', 'belgium_export_mwh', 'belgium_import_mwh']

All datasets processed successfully!


### Merging Datasets
* Converted `start_date` in the prices dataset to `datetime64[ns]` to match other datasets. 
* Merged all datasets (prices, generation, consumption, cross-border) on `start_date` using a left join, preserving all price records. 
* Results in a unified dataset with 19,056 rows and 56 columns, saved as `cleaned_merged_dataset.csv`.

In [65]:
# Convert prices' start_date to match other datasets
datasets["prices"]["start_date"] = pd.to_datetime(
    datasets["prices"]["start_date"],
    format="%b %d, %Y %I:%M %p"  
)

In [67]:
print(f"Prices dtype after conversion: {datasets['prices']['start_date'].dtype}")


Prices dtype after conversion: datetime64[ns]


In [69]:
merged = (
    datasets["prices"]
    .merge(datasets["generation"], on="start_date", how="left", suffixes=("", "_gen"))
    .merge(datasets["consumption"], on="start_date", how="left")
    .merge(datasets["cross_border"], on="start_date", how="left")
)

In [71]:
print(f"Merged data shape: {merged.shape}")
print(f"Columns: {merged.columns.tolist()}")

Merged data shape: (19056, 56)
Columns: ['start_date', 'end_date', 'germanyluxembourg_mwh', 'delu_neighbours_mwh', 'belgium_mwh', 'denmark_1_mwh', 'denmark_2_mwh', 'france_mwh', 'netherlands_mwh', 'norway_2_mwh', 'austria_mwh', 'poland_mwh', 'sweden_4_mwh', 'switzerland_mwh', 'czech_republic_mwh', 'deatlu_mwh', 'northern_italy_mwh', 'slovenia_mwh', 'hungary_mwh', 'biomass_mwh', 'hydropower_mwh', 'wind_offshore_mwh', 'wind_onshore_mwh', 'photovoltaics_mwh', 'other_renewable_mwh', 'nuclear_mwh', 'lignite_mwh', 'hard_coal_mwh', 'fossil_gas_mwh', 'hydro_pumped_storage_mwh_x', 'other_conventional_mwh', 'residual_load_mwh', 'hydro_pumped_storage_mwh_y', 'net_export_mwh', 'netherlands_export_mwh', 'netherlands_import_mwh', 'switzerland_export_mwh', 'switzerland_import_mwh', 'denmark_export_mwh', 'denmark_import_mwh', 'czech_republic_export_mwh', 'czech_republic_import_mwh', 'luxembourg_export_mwh', 'luxembourg_import_mwh', 'sweden_export_mwh', 'sweden_import_mwh', 'austria_export_mwh', 'austr

In [73]:
import pandas as pd


# Drop the 'end_date' column
merged = merged.drop(columns=['end_date'])
print("Dropped 'end_date' column.")

# Rename hydro pumped storage columns
merged = merged.rename(columns={
    'hydro_pumped_storage_mwh_x': 'hydro_pumped_gen_mwh',
    'hydro_pumped_storage_mwh_y': 'hydro_pumped_cons_mwh'
})
print("Renamed hydro pumped storage columns.")

# Check missing values in key columns
key_columns = ['germanyluxembourg_mwh', 'residual_load_mwh', 'net_export_mwh']
missing_percentages = merged[key_columns].isnull().mean() * 100
print("Missing percentages in key columns:\n", missing_percentages)

# Handle missing values
if missing_percentages.max() < 5:
    merged = merged.dropna(subset=key_columns)
    print("Dropped rows with missing values in key columns.")
else:
    merged[key_columns] = merged[key_columns].interpolate(method='linear')
    print("Interpolated missing values in key columns.")

# Verify no missing values remain
print("Missing values after handling:\n", merged[key_columns].isnull().sum())

# Ensure 'start_date' is datetime
merged['start_date'] = pd.to_datetime(merged['start_date'], errors='coerce')
assert pd.api.types.is_datetime64_any_dtype(merged['start_date']), "start_date is not datetime"
print("Confirmed 'start_date' is datetime.")

# Convert numerical columns to float
for col in key_columns + ['hydro_pumped_gen_mwh', 'hydro_pumped_cons_mwh']:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')
print("Numerical columns converted to float.")

# Save the cleaned dataset
merged.to_csv('cleaned_merged_dataset.csv', index=False)
print("Saved cleaned dataset to 'cleaned_merged_dataset.csv'.")

Dropped 'end_date' column.
Renamed hydro pumped storage columns.
Missing percentages in key columns:
 germanyluxembourg_mwh     0.000000
residual_load_mwh        97.223971
net_export_mwh           60.054576
dtype: float64
Interpolated missing values in key columns.
Missing values after handling:
 germanyluxembourg_mwh     0
residual_load_mwh         2
net_export_mwh           17
dtype: int64
Confirmed 'start_date' is datetime.
Numerical columns converted to float.
Saved cleaned dataset to 'cleaned_merged_dataset.csv'.


In [75]:

import pandas as pd

# Drop 'residual_load_mwh' due to excessive missing data
merged = merged.drop(columns=['residual_load_mwh'])

# Impute missing values in 'net_export_mwh' with the median
median_net_export = merged['net_export_mwh'].median()
merged['net_export_mwh'] = merged['net_export_mwh'].fillna(median_net_export)

# Drop rows with any remaining missing values in key columns
key_columns = ['germanyluxembourg_mwh', 'net_export_mwh']
merged = merged.dropna(subset=key_columns)

# Verify no missing values remain
print("Missing values after final cleaning:\n", merged[key_columns].isnull().sum())

# Save the updated cleaned dataset
merged.to_csv('cleaned_merged_dataset.csv', index=False)
print("Updated cleaned dataset saved to 'cleaned_merged_dataset.csv'.")

Missing values after final cleaning:
 germanyluxembourg_mwh    0
net_export_mwh           0
dtype: int64
Updated cleaned dataset saved to 'cleaned_merged_dataset.csv'.
